In [1]:
from typing import Dict, Iterable, Optional

import numpy as np
import torch
from torch.distributions import Normal, Poisson
from torch.distributions import kl_divergence as kld
from torch import tensor
from complementary_models import HALOVIR as HALOVI
from complementary_models import HALOVAER as HALOVAE
import scanpy as sc
import scvi
import pandas as pd
# torch.autograd.set_detect_anomaly(True)

Global seed set to 0


In [2]:
### test whole data with RNA only 
adata_multi = sc.read_h5ad("halo/E18_mouse_Brain/multiomic.h5ad")
adata_multi.obs["batch_id"] = 1
adata_multi.var["modality"] =adata_multi.var["feature_types"]
adata_mvi = scvi.data.organize_multiome_anndatas(adata_multi)

df_meta= pd.read_csv("halo/E18_mouse_Brain/RNA/metadata.tsv",sep = "\t",index_col=0)
bins = df_meta.binned.unique()
times = {}
index = 0
for bin in sorted(bins):
    times[bin] = index
    index += 1

def add_time(row, times):
    timestamp = times[row.binned]
    return timestamp

df_meta['time_key'] = df_meta.apply(lambda row: add_time(row, times), axis=1)

newindex = []

for idx, row in df_meta.iterrows():
    newindex.append(idx+"_paired")

df_meta['Id'] = newindex    

df_meta_sub = df_meta[["Id", 'latent_time']]

df_meta_sub.set_index("Id", inplace=True)
adata_mvi.obs = adata_mvi.obs.join(df_meta_sub, how="inner")
sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))

In [3]:

HALOVI.setup_anndata(adata_mvi, batch_key="modality", time_key='latent_time')
model = HALOVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene Expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum()
)

In [4]:

## Train Multi-Vi and GLUE
model.module.set_train_params(expr_train=True, acc_train=True)
model.train(max_epochs=20)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=20 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 20/20: 100%|██████████| 20/20 [00:52<00:00,  2.61s/it, loss=1.09e+05, v_num=1]


In [6]:
latent_expr, latent_acc,latent_expr_dep, latent_atac_dep, latent_expr_indep, latent_atac_indep, times  = model.get_latent_representation()

In [6]:
z_multi_VI = 1/2 * (latent_expr+ latent_acc)
z_multi_VI.shape

(3365, 10)

In [24]:
from complementary_models import torch_infer_nonsta_dir

length=z_multi_VI.shape[1]
scores = []
z_multi_acc = z_multi_VI[: ,:5]
z_multi_expr = z_multi_VI[: ,5:]

z_multi_acc = torch.tensor(z_multi_acc).to('cuda')
z_multi_expr = torch.tensor(z_multi_expr).to('cuda')
# times = torch.tensor(times).to('cuda')
score1, _, _ = torch_infer_nonsta_dir(z_multi_acc, z_multi_expr, times)
score2, _, _ = torch_infer_nonsta_dir(z_multi_expr, z_multi_acc, times)

# for i in range(0, length, 2):
#     z1 =  torch.tensor(z_multi_VI[:, i]).to('cuda')
#     z2 = torch.tensor(z_multi_VI[:, i+1]).to('cuda')
#     print(z1.shape, z2.shape)
#     score, _, _ = torch_infer_nonsta_dir(z1, z2, times)
#     scores.append(score)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

/tmp/ipykernel_6350/1304969321.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z_multi_acc = torch.tensor(z_multi_acc).to('cuda')
/tmp/ipykernel_6350/1304969321.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z_multi_expr = torch.tensor(z_multi_expr).to('cuda')


score1 0.0646438486580749 and score2 0.0646493024093862


tensor(-5.4538e-06, device='cuda:0', dtype=torch.float64)

In [11]:
from complementary_models import torch_infer_nonsta_dir
glue_acc =  torch.tensor(latent_acc).to('cuda')
glue_exp = torch.tensor(latent_expr).to('cuda')
times = torch.tensor(times).to('cuda')

score1, _, _ = torch_infer_nonsta_dir(glue_acc, glue_exp, times)
score2, _, _ = torch_infer_nonsta_dir(glue_acc, glue_exp, times)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

score1 0.06458684055899336 and score2 0.06458684055899336


tensor(0., device='cuda:0', dtype=torch.float64)

In [13]:
glue_acc.shape

torch.Size([3365, 10])

In [33]:
#### testPCA and SVD
print((adata_mvi.var["feature_types"]=="Gene Expression").sum())
print((adata_mvi.var["feature_types"]!="Gene Expression").sum())

14583
123883


In [ ]:
## gene expression 14583
## atac peak 123883

In [32]:
adata_mvi.X.shape

(3365, 138466)

In [34]:
gene_expr = adata_mvi.X[:, :14583]
gene_peak = adata_mvi.X[:, 14583:]
print(gene_expr.shape, gene_peak.shape)

(3365, 14583) (3365, 123883)


In [40]:
times = adata_mvi.obs["latent_time"]
times = times.to_numpy()

In [42]:
type(times)
times.shape

(3365,)

In [43]:
Gene_PCA = sc.pp.pca(gene_expr, n_comps=10)

In [45]:
Peak_SVD = sc.pp.pca(gene_expr, n_comps=10, svd_solver='arpack')

In [46]:
Gene_PCA.shape

(3365, 10)

In [52]:
times.shape

(3365, 1)

In [53]:
Peak_SVD =  torch.tensor(Peak_SVD).to('cuda')
Gene_PCA = torch.tensor(Gene_PCA).to('cuda')
times = torch.tensor(times).to('cuda')

score1, _, _ = torch_infer_nonsta_dir(Peak_SVD, Gene_PCA, times)
score2, _, _ = torch_infer_nonsta_dir(Gene_PCA, Peak_SVD, times)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

/tmp/ipykernel_6350/2506762614.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Peak_SVD =  torch.tensor(Peak_SVD).to('cuda')
/tmp/ipykernel_6350/2506762614.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Gene_PCA = torch.tensor(Gene_PCA).to('cuda')


score1 0.06462240792863325 and score2 0.06462297989043214


tensor(-5.7196e-07, device='cuda:0', dtype=torch.float64)

In [7]:
### running clustering for evaluation
## GLUE
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI


latent_rep = np.concatenate((latent_expr, latent_acc), axis=1)
adata_mvi.obsm["latent_rep"] = latent_rep
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])

print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.44335601678455655, NMI 0.545887147803308


In [8]:
### running clustering 
### MULTIVI
z_multi_VI = 1/2 * (latent_expr+ latent_acc)
adata_mvi.obsm["latent_rep"] = z_multi_VI
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])

print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.38418582773868637, NMI 0.4977051826934314


In [9]:
gene_expr = adata_mvi.X[:, :14583]
gene_peak = adata_mvi.X[:, 14583:]
Gene_PCA = sc.pp.pca(gene_expr, n_comps=10)

adata_mvi.obsm["latent_rep"] = Gene_PCA
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.3072549237229667, NMI 0.40330159694540185


In [10]:
Peak_SVD = sc.pp.pca(gene_peak, n_comps=10, svd_solver='arpack')
adata_mvi.obsm["latent_rep"] = Peak_SVD
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['celltype'], adata_mvi.obs['leiden_latent'])
print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.1419958829095547, NMI 0.25848174788167677
